In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format, dayofweek, monotonically_increasing_id
from pyspark.sql.types import TimestampType

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1674468952979_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_data = "s3a://udacity-dend/"
output_data = "s3a://naomi-udacity-data-lake/"

spark = SparkSession.builder.config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# get filepath to song data file
song_data = os.path.join(input_data, 'song_data/A/A/A/*.json')

# read song data file
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df.head(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist_id='ARTC1LV1187B9A4858', artist_latitude=51.4536, artist_location="Goldsmith's College, Lewisham, Lo", artist_longitude=-0.01802, artist_name='The Bonzo Dog Band', duration=301.40036, num_songs=1, song_id='SOAFBCP12A8C13CC7D', title='King Of Scurf (2007 Digital Remaster)', year=1972), Row(artist_id='ARA23XO1187B9AF18F', artist_latitude=40.57885, artist_location='Carteret, New Jersey', artist_longitude=-74.21956, artist_name='The Smithereens', duration=192.522, num_songs=1, song_id='SOKTJDS12AF72A25E5', title='Drown In My Own Tears (24-Bit Digitally Remastered 04)', year=0), Row(artist_id='ARSVTNL1187B992A91', artist_latitude=51.50632, artist_location='London, England', artist_longitude=-0.12714, artist_name='Jonathan King', duration=129.85424, num_songs=1, song_id='SOEKAZG12AB018837E', title="I'll Slap Your Face (Entertainment USA Theme)", year=2001)]

In [6]:
songs_table = df.select('song_id', 'title', 'artist_id', 'year', 'duration')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
songs_table.head(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(song_id='SOAFBCP12A8C13CC7D', title='King Of Scurf (2007 Digital Remaster)', artist_id='ARTC1LV1187B9A4858', year=1972, duration=301.40036), Row(song_id='SOKTJDS12AF72A25E5', title='Drown In My Own Tears (24-Bit Digitally Remastered 04)', artist_id='ARA23XO1187B9AF18F', year=0, duration=192.522), Row(song_id='SOEKAZG12AB018837E', title="I'll Slap Your Face (Entertainment USA Theme)", artist_id='ARSVTNL1187B992A91', year=2001, duration=129.85424)]

In [8]:
songs_table.write.partitionBy('year','artist_id').mode('overwrite').\
        parquet(output_data + 'songs_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# extract columns to create artists table
artists_table = df.select('artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
artists_table.head(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist_id='ARTC1LV1187B9A4858', artist_name='The Bonzo Dog Band', artist_location="Goldsmith's College, Lewisham, Lo", artist_latitude=51.4536, artist_longitude=-0.01802), Row(artist_id='ARA23XO1187B9AF18F', artist_name='The Smithereens', artist_location='Carteret, New Jersey', artist_latitude=40.57885, artist_longitude=-74.21956), Row(artist_id='ARSVTNL1187B992A91', artist_name='Jonathan King', artist_location='London, England', artist_latitude=51.50632, artist_longitude=-0.12714)]

In [11]:
# write artists table to parquet files
artists_table.write.mode('overwrite').parquet(output_data + 'artists_table.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# get filepath to log data file
log_data = os.path.join(input_data, 'log_data/2018/11/*.json')

# read log data file
df = spark.read.json(log_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# filter by actions for song plays
df = df.where(df.page == 'NextSong')
df.head(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26'), Row(artist='The Prodigy', auth='Logged In', firstName='Ryan', gender='M', itemInSession=1, lastName='Smith', length=260.07465, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='The Big Gundown', status=200, ts=1542242481796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26'), Row(artist='Train', auth='Logged In', 

In [14]:
# extract columns for users table
users_table = df.select(df.userId.alias('user_id'), df.firstName.alias('first_name'), df.lastName.alias('last_name'), 'gender', 'level').distinct()
users_table.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(user_id='26', first_name='Ryan', last_name='Smith', gender='M', level='free')]

In [15]:
# write users table to parquet files
users_table.write.mode('overwrite').parquet(os.path.join(output_data + 'users_table.parquet'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# create timestamp column from original timestamp column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x / 1000), TimestampType())
df = df.withColumn("start_time", get_timestamp('ts'))

df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)

In [17]:
df.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist='Harmonia', auth='Logged In', firstName='Ryan', gender='M', itemInSession=0, lastName='Smith', length=655.77751, level='free', location='San Jose-Sunnyvale-Santa Clara, CA', method='PUT', page='NextSong', registration=1541016707796.0, sessionId=583, song='Sehr kosmisch', status=200, ts=1542241826796, userAgent='"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"', userId='26', start_time=datetime.datetime(2018, 11, 15, 0, 30, 26, 796000))]

In [18]:
df = df.withColumn('hour', hour('start_time')) \
    .withColumn('day', dayofmonth('start_time')) \
    .withColumn('week', weekofyear('start_time')) \
    .withColumn('month', month('start_time')) \
    .withColumn('year', year('start_time')) \
    .withColumn('weekday', dayofweek('start_time')) 
time_table = df.select('start_time', 'weekday', 'year', 'month', 'week', 'day', 'hour').distinct()
time_table.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(start_time=datetime.datetime(2018, 11, 15, 14, 9, 23, 796000), weekday=5, year=2018, month=11, week=46, day=15, hour=14)]

In [19]:
# write time table to parquet files partitioned by year and month
time_table.write.partitionBy('year','month').mode('overwrite').\
    parquet(os.path.join(output_data, 'time_table.parquet'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# read in song data to use for songplays table
song_df = spark.read.parquet(os.path.join(output_data, 'songs_table.parquet'))
artist_df = spark.read.parquet(os.path.join(output_data, 'artists_table.parquet'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
song_df.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(song_id='SOKTJDS12AF72A25E5', title='Drown In My Own Tears (24-Bit Digitally Remastered 04)', duration=192.522, year=0, artist_id='ARA23XO1187B9AF18F')]

In [22]:
artist_df.head(1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(artist_id='ARTC1LV1187B9A4858', artist_name='The Bonzo Dog Band', artist_location="Goldsmith's College, Lewisham, Lo", artist_latitude=51.4536, artist_longitude=-0.01802)]

In [23]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

In [25]:
# extract columns from joined song and log datasets to create songplays table 
#songplays_table = df.join(song_df,
#                          on=[
#                              df.artist == song_df.artistName,
#                              df.song == song_df.title,
#                              df.length == song_df.duration
#                          ],
#                          how='inner')
songplays_table = df.join(song_df,
                          on=[df.song == song_df.title,
                              df.length == song_df.duration],
                          how='inner') \
                    .join(artist_df,
                          on=[df.artist == artist_df.artist_name,
                              song_df.artist_id == artist_df.artist_id],
                          how='inner') \
                    .drop(song_df.artist_id) \
                    .drop(song_df.year) \
                    .select('start_time', df.userId.alias('user_id'), 'level', 'song_id',
                            'artist_id', df.sessionId.alias('session_id'), 'location',
                            df.userAgent.alias('user_agent'), df.year, df.month) \
                    .withColumn("songplay_id", monotonically_increasing_id())

songplays_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- user_id: string (nullable = true)
 |-- level: string (nullable = true)
 |-- song_id: string (nullable = true)
 |-- artist_id: string (nullable = true)
 |-- session_id: long (nullable = true)
 |-- location: string (nullable = true)
 |-- user_agent: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- songplay_id: long (nullable = false)

In [26]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.partitionBy('year','month').mode('overwrite').\
    parquet(os.path.join(output_data, 'songplays_table.parquet'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…